# Neural Machine Translation
### Sequence to Sequence Network with Attention


In [6]:
# Import Libraries
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader, RandomSampler

# Device Agnostic Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### 00. Loading Data Files

In [7]:
SOS_token = 0
EOS_token = 1

class Language:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # For counting SOS and EOS token

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [87]:
# Lowercase, rim and remove non-letter characters
def normalizeString(s):
    s = s.lower().strip()
    # Add spaces around punctuation
    s = re.sub(r"([.!?।])", r" \1", s)
    # Retain Devanagari characters, Latin characters, English numbers, and punctuation
    s = re.sub(r"[^\u0900-\u097Fa-zA-Z0-9.!?']+", r" ", s)
    return s


In [88]:
import csv

def readLangs(file_path, lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and parse CSV content
    with open(file_path, encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        lines = list(reader)

    # Normalize each pair
    pairs = [[normalizeString(s) for s in l] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Language(lang2)
        output_lang = Language(lang1)
    else:
        input_lang = Language(lang1)
        output_lang = Language(lang2)

    return input_lang, output_lang, pairs

In [94]:
MAX_LENGTH = 24

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [95]:
def prepareData(file_path, lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(file_path, lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [98]:
input_lang, output_lang, pairs = prepareData('eng_nep_nmt.csv', 'eng', 'nep', False)

Reading lines...
Read 160259 sentence pairs
Trimmed to 119560 sentence pairs
Counting words...
Counted words:
eng 44799
nep 119592


In [99]:
random.choice(pairs)

['tension started on monday from kathmandu on september 9 .',
 'चैत २३ गते बेलुकीदेखि काठमाण्डौमा तनाव शुरु भएको थियो ।']